## Global API - RAG
Global API（任意のCollectionに対してではなくh2oGPTe本体に対するAPI）を用いたh2oGPTeに対する操作とRAGの実施

In [1]:
from h2ogpte import H2OGPTE

import json

In [4]:
!ls

Collection-specific-API.ipynb     _secret
Collection-specific-API_RAG.ipynb data
Global-API_RAG.ipynb              readme.md


In [6]:
with open('_secret') as f:
    key = json.load(f)

In [8]:
client = H2OGPTE(
    address='https://h2ogpte.internal.dedicated.h2o.ai',
    api_key=key['global_internal-dedicated-0726'],
)

client

Please install the correct version of H2OGPTE with `pip install h2ogpte==1.5.8`.
You can enable strict version checking by passing strict_version_check=True.


### Collectionの作成

In [16]:
client.list_embedding_models()

['BAAI/bge-large-en-v1.5', 'BAAI/bge-m3', 'hkunlp/instructor-large', 'off']

In [20]:
# Create a new collection
collection_id = client.create_collection(
    name='resume-analyzer',
    description='職務経歴書に対するRAG',
    embedding_model='BAAI/bge-m3',
    prompt_template_id='16095e26-6876-410f-a934-3c0651a21e4c'    # h2oGPTe UI（チャットの設定）から取得
)
collection_id

'17c3c8c8-f7cb-4f6d-9c50-c369d4f41518'

create_collection(): https://h2oai.github.io/h2ogpte/h2ogpte.html#h2ogpte.h2ogpte.H2OGPTE.create_collection

### Documentのアップロードと知識データベース化（chunking, indexing etc）

In [21]:
!ls data

resume_TaroShonan.docx


In [22]:
# Upload documents
with open('data/resume_TaroShonan.docx', 'rb') as f:
    upload_file = client.upload('resume_TaroShonan.docx', f)

upload_file

'112e2257-b266-4bc8-8a7e-1d49e3619e89'

upload(): https://h2oai.github.io/h2ogpte/h2ogpte.html#h2ogpte.h2ogpte.H2OGPTE.upload

In [23]:
# Ingest documents (Creates previews, chunks and embeddings)
client.ingest_uploads(collection_id, [upload_file])

Job(id='e152ac53-264f-47aa-81bf-307b124c2ef7', name='Adding documents', passed=1.0, failed=0.0, progress=1.0, completed=True, canceled=False, date=datetime.datetime(2024, 7, 26, 2, 27, 38, tzinfo=TzInfo(UTC)), kind=<JobKind.IngestUploadsJob: 'IngestUploadsJob'>, statuses=[JobStatus(id='c4184dc7ee4847c0b02f20562af230aa', status='Indexing done.'), JobStatus(id='8e32806b56614437a80c4004ffc916b5', status='Collecting done.')], errors=[], last_update_date=datetime.datetime(2024, 7, 26, 2, 27, 45, tzinfo=TzInfo(UTC)), duration='7s', duration_seconds=7.0)

ingest_uploads(): https://h2oai.github.io/h2ogpte/h2ogpte.html#h2ogpte.h2ogpte.H2OGPTE.ingest_uploads

In [27]:
# Collection内のDocument
client.list_documents_in_collection(collection_id, offset=0, limit=999)

[DocumentInfo(id='66191b88-dfe5-466b-9ea3-9768c5098739', username='yuki.shimada@h2o.ai', name='resume_TaroShonan.pdf', type='PDF', size=115555, page_count=3, pii_settings=None, connector='Upload', uri=None, original_type='Word', meta_data_dict={}, status='completed', updated_at=datetime.datetime(2024, 7, 26, 2, 27, 45, 271093, tzinfo=TzInfo(UTC)))]

### チャット

In [29]:
client.get_llm_names()

['mistralai/Mixtral-8x7B-Instruct-v0.1',
 'mistralai/Mistral-7B-Instruct-v0.3',
 'meta-llama/Meta-Llama-3.1-8B-Instruct',
 'meta-llama/Meta-Llama-3.1-70B-Instruct',
 'OpenGVLab/InternVL-Chat-V1-5',
 'h2oai/h2o-danube3-4b-chat',
 'mistral-medium-latest',
 'claude-3-5-sonnet-20240620',
 'claude-3-haiku-20240307',
 'gpt-35-turbo-1106',
 'gpt-4-1106-preview',
 'gpt-4o',
 'gemini-1.5-pro-latest']

In [24]:
# Create a chat session
chat_session_id = client.create_chat_session(collection_id)
chat_session_id

'9d5362e4-8124-4985-99ab-1b3113448419'

In [28]:
%%time
with client.connect(chat_session_id) as session:
    reply = session.query(
        '誰の職務経歴書になりますか？また、この人の得意な業務を教えて下さい。',
        llm='mistralai/Mixtral-8x7B-Instruct-v0.1',
        rag_config={"rag_type": "rag"},   # (Normal) RAG 
    )
    print(reply.content)

この職務経歴書は「湘南 太郎」という人物のものです。この人は、5年以上の経験を持ったデータ活用領域のデータサイエンティストやコンサルタントであり、顧客の課題理解、分析やモデル作成、プロジェクトマネジメント、顧客マネジメントまで幅広く対応できます。また、元々大学でマーケティングを勉強し、その後マーケティングのコンサルティング会社で勤務していたため、関連業務に明るいとのことです。

この人の得意な業務は、以下の通りです。

* 顧客へのヒアリングから要件定義、プロジェクトマネジメントまでの経験
* データ分析と顧客への結果報告
* 5名規模のチームマネジメント経験
* マーケティング関連業務の知識

さらに、この人はPython、AWS、画像処理、統計解析、データビジュアライゼーションなどの技術スキルを持っています。
CPU times: user 9.3 ms, sys: 4.11 ms, total: 13.4 ms
Wall time: 8.35 s


In [30]:
%%time
with client.connect(chat_session_id) as session:
    reply = session.query(
        '書かれている技術スキルをあげて下さい。',
        llm='mistralai/Mixtral-8x7B-Instruct-v0.1',
        rag_config={"rag_type": "rag"},   # (Normal) RAG 
    )
    print(reply.content)

Taro Shonanの技術スキルは、Python, AWS, 画像処理, 統計解析, データビジュアライゼーション, R, SQL, 自然言語処理, 生成AI, Linux, Docker, Githubです。
CPU times: user 6.09 ms, sys: 2.9 ms, total: 8.99 ms
Wall time: 3.83 s


In [38]:
%%time
with client.connect(chat_session_id) as session:
    reply = session.query(
        '書かれている技術スキルをあげて下さい。',
        llm='mistralai/Mixtral-8x7B-Instruct-v0.1',
        rag_config={"rag_type": "rag"},   # (Normal) RAG 
        llm_args=dict(
            response_format='json_object',
            guided_json={
                '$schema': 'http://json-schema.org/draft-07/schema#',
                'type': 'object',
                'properties': {'count': {'type': 'integer'}},
                'required': [
                    'count',
                ],
            },
        ),
    )
    print(reply.content)

{"count": 14}
CPU times: user 5.07 ms, sys: 2.26 ms, total: 7.33 ms
Wall time: 2.89 s


In [35]:
reply

ChatMessage(id='971520be-a904-4e8a-bba5-3879df41c2ca', content='{"count": 12}', reply_to='f1050c4a-aeaa-4a8e-a67f-6f5fd742f379', votes=0, created_at=datetime.datetime(2024, 7, 26, 11, 48, 2, 790085), type_list=[], error=None)

In [39]:
%%time
with client.connect(chat_session_id) as session:
    reply = session.query(
        '書かれている技術スキルをあげて下さい。',
        llm='mistralai/Mixtral-8x7B-Instruct-v0.1',
        rag_config={"rag_type": "rag"},   # (Normal) RAG 
        llm_args=dict(
            response_format='json_object',
        ),
    )
    print(reply.content)

{"skills": ["Python", "AWS", "\u753b\u50cf\u51e6\u7406", "\u7d71\u8a08\u89e3\u6790", "\u30c7\u30fc\u30bf\u30d3\u30b8\u30e5\u30a2\u30e9\u30a4\u30bc\u30fc\u30b7\u30e7\u30f3", "R", "SQL", "\u81ea\u7136\u8a00\u8a9e\u51e6\u7406", "\u751f\u6210AI", "Linux", "Docker", "Github"]}
CPU times: user 6.24 ms, sys: 2.34 ms, total: 8.58 ms
Wall time: 3.74 s


In [45]:
json.loads(reply.content)

{'skills': ['Python',
  'AWS',
  '画像処理',
  '統計解析',
  'データビジュアライゼーション',
  'R',
  'SQL',
  '自然言語処理',
  '生成AI',
  'Linux',
  'Docker',
  'Github']}

query(): https://h2oai.github.io/h2ogpte/h2ogpte.html#h2ogpte.session.Session.query

llm_args
- temperature (float, default: 0) — The value used to modulate the next token probabilities. Most deterministic: 0, Most creative: 1 
- top_k (int, default: 1) — The number of highest probability vocabulary tokens to keep for top-k-filtering. 
- top_p (float, default: 1.0) — If set to float < 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation. 
- seed (int, default: 0) — The seed for the random number generator when sampling during generation (if temp>0 or top_k>1 or top_p<1), seed=0 picks a random seed. 
- repetition_penalty (float, default: 1.07) — The parameter for repetition penalty. 1.0 means no penalty. 
- max_new_tokens (int, default: 1024) — Maximum number of new tokens to generate. This limit applies to each (map+reduce) step during summarization and each (map) step during extraction. 
- min_max_new_tokens (int, default: 512) — minimum value for max_new_tokens when auto-adjusting for content of prompt, docs, etc. 
- response_format (str, default: “text”) — Output type, one of [“text”, “json_object”, “json_code”]. 
- guided_json (str, default: “”) — If specified, the output will follow the JSON schema. 
- guided_regex (str, default: “”) — If specified, the output will follow the regex pattern. 
- guided_choice (Optional[List[str]], default: None — If specified, the output will be exactly one of the choices. 
- guided_grammar (str, default: “”) — If specified, the output will follow the context free grammar. 
- guided_whitespace_pattern (str, default: “”) — If specified, will override the default whitespace pattern for guided json decoding. 
- enable_vision (str, default: “auto”) - Controls vision mode, send images to the LLM in addition to text chunks. Only if have models that support vision, use get_vision_capable_llm_names() to see list. One of [“on”, “off”, “auto”]. 
- visible_vision_models (List[str], default: [“auto”]) - Controls which vision model to use when processing images. Use get_vision_capable_llm_names() to see list. Must provide exactly one model. [“auto”] for automatic.

In [50]:
%%time
with client.connect(chat_session_id) as session:
    reply = session.query(
        '書かれているマネジメント関連のスキルをあげて下さい。',
        llm='mistralai/Mixtral-8x7B-Instruct-v0.1',
        rag_config={"rag_type": "rag"},   # (Normal) RAG 
        llm_args=dict(
            response_format='json_object',
        ),
    )
    print(reply.content)

{"management_skills": ["\u30c1\u30fc\u30e0\u7ba1\u7406", "\u30d7\u30ed\u30b8\u30a7\u30af\u30c8\u7ba1\u7406", "\u6226\u7565\u7acb\u6848", "\u9867\u5ba2\u5bfe\u5fdc", "\u30c1\u30fc\u30e0\u30e1\u30f3\u30d0\u30fc\u306e\u63a1\u7528\u3001\u30c8\u30ec\u30fc\u30cb\u30f3\u30b0\u3001\u304a\u3088\u3073\u6307\u5c0e", "\u30c1\u30fc\u30e0\u30e1\u30f3\u30d0\u30fc\u306e\u696d\u7e3e\u8a55\u4fa1\u3068\u30d5\u30a3\u30fc\u30c9\u30d0\u30c3\u30af\u306e\u63d0\u4f9b", "\u30c1\u30fc\u30e0\u306e\u30b9\u30ad\u30eb\u958b\u767a\u3068\u30ad\u30e3\u30ea\u30a2\u6210\u9577\u306e\u30b5\u30dd\u30fc\u30c8", "\u30c7\u30fc\u30bf\u53ce\u96c6\u3001\u4fdd\u5b58\u3001\u7ba1\u7406\u306e\u305f\u3081\u306e\u30dd\u30ea\u30b7\u30fc\u3068\u30d7\u30ed\u30bb\u30b9\u306e\u7b56\u5b9a", "\u30c7\u30fc\u30bf\u306e\u54c1\u8cea\u7ba1\u7406\u3068\u30ac\u30d0\u30ca\u30f3\u30b9\u306e\u78ba\u4fdd", "\u30c7\u30fc\u30bf\u6226\u7565\u306e\u7b56\u5b9a", "\u30c7\u30fc\u30bf\u30a4\u30f3\u30d5\u30e9\u306e\u8a2d\u8a08\u3068\u5b9f\u88c5", "\u30af\u30e9\u

In [51]:
json.loads(reply.content)

{'management_skills': ['チーム管理',
  'プロジェクト管理',
  '戦略立案',
  '顧客対応',
  'チームメンバーの採用、トレーニング、および指導',
  'チームメンバーの業績評価とフィードバックの提供',
  'チームのスキル開発とキャリア成長のサポート',
  'データ収集、保存、管理のためのポリシーとプロセスの策定',
  'データの品質管理とガバナンスの確保',
  'データ戦略の策定',
  'データインフラの設計と実装',
  'クライアントの従業員に対するデータリテラシーや分析スキルのトレーニング',
  'データプロジェクトの計画、実行、進捗管理',
  'プロジェクトの成果物の品質保証と納品',
  'コンサルティング',
  '営業']}